In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.item_processing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = ItemFeatureProcessing(config)

# Answer Single Question ITEM Level Processing

In [ ]:
df_item = features_class.df_item
self = features_class

In [ ]:
single_question_mask = ((self._df_item['type'] == 'SingleQuestion')
                        & (self._df_item['n_answers'] > 2)
                        & (self._df_item['is_filtered_combobox'] == False)
                        & (pd.isnull(self._df_item['cascade_from_question_id'])))

In [ ]:
df_item[single_question_mask].groupby(['variable_name'])['value'].unique()

In [ ]:
feature_name = 'f__single_question'
score_name = 's__single_question'
df = df_item[single_question_mask].copy()
# Select only those variables that have at least three distinct values and more than one hundred records

variables =  self.filter_variable_name_by_frequency(df, 'value', frequency=100, min_unique_values=3)


In [ ]:
for var in variables:

    mask = (df['variable_name']==var) 
    unique_values = df[mask]['value'].nunique()
    entropy_df = df[mask].groupby('responsible')['value'].apply(calculate_entropy,unique_values=unique_values, min_record_sample=5).copy()
    entropy_df = entropy_df.reset_index()
    entropy_df = entropy_df[~pd.isnull(entropy_df['value'])]
    
    if entropy_df.shape[0] > 0:
        entropy_df.sort_values('value', inplace=True, ascending=False)

        median_value = entropy_df['value'].median()
        entropy_df[score_name] = entropy_df['value'].apply(lambda x: 1 if x< median_value-50/100*median_value else 0)

        anomaly_df = entropy_df[entropy_df[score_name] == 1]
        no_anomaly_df = entropy_df[entropy_df[score_name] == 0]
        plt.bar(no_anomaly_df['responsible'], no_anomaly_df['value'], color='blue', label='No Anomaly')
        plt.bar(anomaly_df['responsible'], anomaly_df['value'], color='red', label='Anomaly')
        plt.xticks(rotation=90)
        plt.title(var)
        plt.show()



In [ ]:
for var in variables:
    mask = (df['variable_name']==var)
    unique_values = df[mask]['value'].nunique()
    entropy_df = df[mask].groupby('responsible')['value'].apply(calculate_entropy,unique_values=unique_values)
    entropy_df = entropy_df.reset_index()
    entropy_df = entropy_df[~pd.isnull(entropy_df['value'])]
    
    if entropy_df.shape[0] > 0:
        entropy_df.sort_values('value', inplace=True, ascending=False)

        median_value = entropy_df['value'].median()

        median_value = entropy_df['value'].median()
        entropy_df[score_name] = entropy_df['value'].apply(lambda x: 1 if x < median_value - 50/100 * median_value else 0)
        df.loc[mask,score_name] = df[mask]['responsible'].map(entropy_df.set_index('responsible')[score_name])

# Answer Single Question Responsible Level Processing

In [ ]:
data = df.groupby(['responsible','variable_name']).agg({score_name: 'mean'})
data = data.reset_index()
entropy_ = data.groupby('responsible')[score_name].mean()
entropy_.plot(kind='bar')
plt.title('entropy__answer_position')
plt.show()